In [13]:
import numpy # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3 # Database
import time

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

In [24]:
def get_table_columns(db, table_name):
    sql = f"""SELECT * FROM {table_name} LIMIT 1"""
    table_values = pandas.read_sql(sql, db)
    return table_values.columns

In [62]:
def get_n_records(db, table_name):
    sql = f"""SELECT COUNT(*) FROM {table_name}"""
    n_rows = pandas.read_sql(sql, db)
    return n_rows['COUNT(*)'].iloc[0]

In [54]:
# Connect to db
# Tables: ascent, grade, method, user
db = sqlite3.connect('input/database.sqlite')
cursor = db.cursor()

In [55]:
# What columns are there?
print(get_table_columns(db, 'ascent'))

Index(['id', 'user_id', 'grade_id', 'notes', 'raw_notes', 'method_id',
       'climb_type', 'total_score', 'date', 'year', 'last_year', 'rec_date',
       'project_ascent_date', 'name', 'crag_id', 'crag', 'sector_id', 'sector',
       'country', 'comment', 'rating', 'description', 'yellow_id', 'climb_try',
       'repeat', 'exclude_from_ranking', 'user_recommended', 'chipped'],
      dtype='object')


In [57]:
# And how many records?
print(get_n_records(db, 'ascent'))

4111877


In [79]:
# Let's only load some important data
#n_records = 1000

start = time.time()
sql = f"""SELECT id, user_id, grade_id, date, year, name FROM ascent"""
ascents = pandas.read_sql(sql, db)
print(ascents.head())
end = time.time()

print(f"Fetched {ascents.shape[0]/1000:.0f}k records in {end-start:.2f}s")

   id  user_id  grade_id       date  year            name
0   2        1        36  918342000  1999  The King And I
1   3        1        36  925509600  1999          vet ej
2   4        1        36  933026400  1999          Mr Big
3   5        1        36  933026400  1999   Tak ska du ha
4   6        1        36  933458400  1999   Korpen flyger
Fetched 1k records in 0.05s


In [71]:
# Let's get the grades too
print("Grades columns:")
print(get_table_columns(db, 'grade'))
print(f"{get_n_records(db, 'grade')} records")

sql = """SELECT * FROM grade"""
grades = pandas.read_sql(sql, db)
print(grades.fra_routes.values)
grades.head(2)
# Seems like they are arranged according to difficulty, yay!

Grades columns:
Index(['id', 'score', 'fra_routes', 'fra_routes_input', 'fra_routes_selector',
       'fra_boulders', 'fra_boulders_input', 'fra_boulders_selector',
       'usa_routes', 'usa_routes_input', 'usa_routes_selector', 'usa_boulders',
       'usa_boulders_input', 'usa_boulders_selector'],
      dtype='object')
83 records
['-' '1' '1a' '1b' '1c' '1+' '2' '2a' '2b' '2c' '2+' '3' '3a' '3a+' '3b'
 '3b+' '3c' '3c+' '3+' '4' '4a' '4a+' '4b' '4b+' '4c' '4c+' '4+' '5' '5a'
 '5a+' '5b' '5b+' '5c' '5c+' '6' '6a' '6a/+' '6a+' '6a+/6b' '6b' '6b/+'
 '6b+' '6b+/6c' '6c' '6c/+' '6c+' '6c+/7a' '7a' '7a/+' '7a+' '7a+/7b' '7b'
 '7b/+' '7b+' '7b+/7c' '7c' '7c/+' '7c+' '7c+/8a' '8a' '8a/+' '8a+'
 '8a+/8b' '8b' '8b/+' '8b+' '8b+/8c' '8c' '8c/+' '8c+' '8c+/9a' '9a'
 '9a/+' '9a+' '9a+/9b' '9b' '9b/+' '9b+' '9b+/9c' '9c' '9c/+' '9c+'
 '9c+/10a']


,id,score,fra_routes,fra_routes_input,fra_routes_selector,fra_boulders,fra_boulders_input,fra_boulders_selector,usa_routes,usa_routes_input,usa_routes_selector,usa_boulders,usa_boulders_input,usa_boulders_selector
0,1,0,-,1,1,-,1,1,3/4,1,1,VB,1,1
1,2,0,1,0,0,1,0,0,,0,0,,0,0


In [76]:
# Let's now get back to the ascents and just use grade_id as an overall grade
ascents.info()
# No missing data, wow! :D
# Let's just match it back to the users then

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4111877 entries, 0 to 4111876
Data columns (total 3 columns):
id          int64
user_id     int64
grade_id    int64
dtypes: int64(3)
memory usage: 94.1 MB


In [83]:
# How many users are in the ascent table?
sql = f"""SELECT user_id FROM ascent"""
user_ids = pandas.read_sql(sql, db).user_id.unique()
user_ids.shape
# Only about half the registered users!

(36034,)

Are there duplicate entries?